In [1]:
from tqdm import tqdm
import argparse
from utils import *
from freebase import *
from propagation import *
import random
import concurrent.futures



parser = argparse.ArgumentParser()
parser.add_argument("--dataset", type=str,
                    default="cwq", help="choose the dataset from {webqsp, cwq}.")
parser.add_argument("--max_length", type=int,
                    default=1024, help="the max length of LLMs output.")
parser.add_argument("--temperature", type=float,
                    default=0., help="the temperature")
parser.add_argument("--llm", type=str,
                    default="llama-3", help="choose base LLM model from {llama, gpt-3.5-turbo, gpt-4}.")
parser.add_argument("--openai_api_key", type=str,
                    default="", help="if the LLM is gpt-3.5-turbo or gpt-4, you need add your own openai api key.")
parser.add_argument('--verbose', action='store_true', help="print LLM input and output.")
args = parser.parse_args(["--verbose"])
# args = parser.parse_args("")


datas, question_string = prepare_dataset(args.dataset)

In [2]:
data = datas[2]
question = data[question_string]
topics = data['topic_entity']
paths = {topics[topic]: {} for topic in topics}
print(question)

What is the predominant religion where the leader is Ovadia Yosef?


In [3]:
# for topic in topics:
for topic in topics:
    topic_name = topics[topic]
    # 1-hop propagation
    relations = get_relations(question, topic, topic_name, args, 3)
    entities = get_entities({topic: topic_name}, relations, topic)
    [paths[topic_name].update({r: {"entities": entities[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, relations, paths[topic_name], 1, args)
    [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]
    # 2-hop propagation
    relations = get_relations_distant(question, topic, topic_name, relations, paths[topic_name], args, 3)
    entities = get_entities_distant(paths[topic_name], relations, topic)
    [paths[topic_name].update({r: {"entities": entities[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, relations, paths[topic_name], 2, args)
    [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]
    # 3-hop propagation
    relations = get_relations_distant(question, topic, topic_name, relations, paths[topic_name], args, 3)
    entities = get_entities_distant(paths[topic_name], relations, topic)
    [paths[topic_name].update({r: {"entities": entities[i]}}) for i, r in enumerate(relations)]
    facts = propagate(question, topic_name, relations, paths[topic_name], 3, args)
    [paths[topic_name][r].update({"fact": facts[i]}) for i, r in enumerate(relations)]
    # # # clean paths
    [paths[topic_name].update({r: paths[topic_name][r]['fact']}) for r in paths[topic_name]]


===================input======================
Given the question, we have the topic of the question and its relations.

question: What is the predominant religion where the leader is Ovadia Yosef?
topic: Ovadia Yosef

Based on the question and its topic, please select the most relevant relations to answer the question from below and just return top 3 relations in a numbered list without explanation.
relations: award.award_winner.awards_won, organization.organization_founder.organizations_founded, people.deceased_person.date_of_death, people.deceased_person.place_of_burial, people.deceased_person.place_of_death, people.person.children, people.person.date_of_birth, people.person.education, people.person.ethnicity, people.person.gender, people.person.nationality, people.person.place_of_birth, people.person.places_lived, people.person.profession, people.person.religion, people.person.spouse_s, people.deceased_person.cause_of_death, base.schemastaging.context_name.official_name, people.dec

In [11]:
paths

{'France': {'location.location.containedby': 'France is located in Europe and Western Europe, which may help narrow down the search for the country bordering France.',
  'location.location.geolocation': 'The geolocation of France (latitude: 47.0, longitude: 2.0) may not be directly relevant to finding the country bordering France, but it could be useful in identifying nearby countries.',
  'location.country.administrative_divisions': 'France has many administrative divisions, but none of them seem to be directly related to Nijmegen or the airport serving it. However, this fact may be useful in identifying regions or areas near the French border that could be relevant to the question.',
  'location.location.containedby->location.statistical_region.population': 'France is located in Europe and Western Europe, which helps narrow down the search for the country bordering France.',
  'location.location.geolocation->location.geocode.latitude': 'France has a geolocation of latitude: 47.0, lon

In [10]:
facts = construct_facts(paths, topics)
prompt = question_prompt.format(facts, question)
response = run_llm(prompt, args.temperature, args.max_length, args.openai_api_key, args.llm, args.verbose)
output = {"question": question, "result": response, "paths": paths}

===================input======================
Based on the given the facts and your own knowledge, please the answer the question as simple as possible and return all the possible answers in a numbered list.

information: Here are some facts about topic France that may related to the question.
1. France is located in Europe and Western Europe, which may help narrow down the search for the country bordering France.
	1.1. France is located in Europe and Western Europe, which helps narrow down the search for the country bordering France.
		1.1.1. The population of Europe and Western Europe at different years is provided, but it does not directly help in finding the country bordering France with an airport serving Nijmegen.
		1.1.2. The same information as fact 1, providing population data for Europe and Western Europe at different years, but not directly relevant to the question.

2. The geolocation of France (latitude: 47.0, longitude: 2.0) may not be directly relevant to finding the co